# ML App to Create a Custom Newsfeed

## Introduction:

Reading is an excellent way to fill in any knowledge gap. It is even better if there is a way to enchance this experience whereby to quickly get a customised newsfeed tailored to the reader's interests. This project/app aims to help frequent readers such as myself to find even more to read or rather, to satisfy curiousities on certain topics. It can be seen as beneficial to have what you want to read without going through the hassle of digging for it. 

Hopefully by the end of this project, a system can be built that is able to understand a tailored taste in news and be able to send them directly to the user.

## Breakdown of the Project:
- Making a supervised training set with the Pocket App.
- Utilising the Pocket API to obtain stories.
- Making use of the Embedly API to extract story bodies.
- Basics of NLP (Natural Language Processing)
- Classifiers such as SVM (Support Vector Machines)
- IFTTT will be integrated with RSS feeds and Google Sheets.
- Setup a system for a daily personal newsletters

## Installing Pocket Chrome Extension for this project:

The Pocket app/extension allows for saving personalised stories to be read later on. The stories can be linked to tags, and these tags can be different to every user. From this, the idea is to use to to tag 'y' for interesting topics while 'n' for non-interesting topics. Usage of the Pocket app/extension to help create a __supervised training__ set for the model.

Setup and example of usage:
1. Open Chrome, and add Pocket app in the Extensions.
2. Open any link of interest.
3. Click on the Pocket extension icon, once red, it means the article or link have been saved.

To begin constructing the supervised dataset:
1. As you go through your day, read your articles.
2. Tag 'y' for interesting ones, and tag 'n' for non-interesting ones.
3. Note: it is better to have lots (100s) of articles to make the model better.

Retrive the save stories:
1. Go to: https://getpocket.com/developer/apps/new
2. Click on 'Create a New App' in the top left corner of the webpage.
3. Input the details and get the API KEY.
4. Be sure to click on all the Permissions (allows for adding, changing and retrieving)
5. Once done, click to submit.
6. Click on 'My Apps' in the top left corner fo the webpage and you will see the custom applications.

After these, move on to the next stage which is the code below.

## 1 Import the required Libraries:

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os

## 2 Setup Authorisations:

This section requires the Pocket API key (consumer key) and a redirect URL. Note that the redirect URL can be anything here. For example, it can be a twitter account  or otherwise. For more information on Redirect URL: https://www.hostgator.com/blog/ways-redirect-website-url/.

In this case, a twitter account was used.

In [2]:
pd.set_option('display.max_colwidth', 200)

# Get Pocket API Key:
def get_API_KEY(APIKEY_fileName):
    currentDirectory = os.getcwd()
    path_to_API_KEY = currentDirectory + '/' + APIKEY_fileName
    API_KEY_file = open(path_to_API_KEY)
    
    API_KEY = []
    for i in API_KEY_file:
        API_KEY.append(i)
    
    return str(API_KEY[0])

In [3]:
pocket_API_key = get_API_KEY('Pocket API Key.txt')
print(pocket_API_key)

89714-63913284a8c8d0a2e346b63d


In [4]:
# Get Redirect URL from text file:
def get_REDIRECT_URL(RURL_fileName):
    currentDirectory = os.getcwd()
    RURL_path = currentDirectory + '/' + RURL_fileName
    URL_file = open(RURL_path)
    
    RedirectURL = []
    for i in URL_file:
        RedirectURL.append(i)
    
    return str(RedirectURL[0])

In [5]:
Redirect_URL_key = get_REDIRECT_URL('TwitterAccount.txt')
print(Redirect_URL_key)

https://www.twitter.com/YSLee65720613


In [6]:
# Authentication Parameters:
auth_params = {'consumer_key': pocket_API_key, 'redirect_uri': Redirect_URL_key}

token = requests.post('https://getpocket.com/v3/oauth/request', data = auth_params)

print('Access code is: ')
print(token.text)

Access code is: 
code=4224e48f-9712-0c9e-66af-c7b19b


#### Next, is to copy the following output link and AUTHORISE it:

If issues are encountered, use the following link to troubleshoot: http://www.jamesfmackenzie.com/getting-started-with-the-pocket-developer-api/#step-3-visit-the-pocket-website-to-authorize-your-app

Note: Keep the link open for the next few steps, otherwise a 403 forbidden error will occur.

In [8]:
# Generate the new output link:
auth_Link = 'https://getpocket.com/auth/authorize?request_token=' + token.text.strip('code=') + '&redirect_uri=https%3A//' + Redirect_URL_key.strip('https://')

print('Copy the following link and authorise: ')
print(auth_Link)


Copy the following link and authorise: 
https://getpocket.com/auth/authorize?request_token=4224e48f-9712-0c9e-66af-c7b19b&redirect_uri=https%3A//www.twitter.com/YSLee65720613


#### This is then followed by Parsing out the access code:

In [9]:
# Parsing out the access vode from token.text string;
access_code = token.text.split('=')[1]

user_params = {'consumer_key': pocket_API_key, 'code': access_code}

user = requests.post('https://getpocket.com/v3/oauth/authorize', data = user_params)

print('User code is: ')
print(user.text)

User code is: 
access_token=c3427703-c4f7-a7cc-4470-502f4a&username=ylee9106%40gmail.com


## 3 Retrieving the Saved Online Articles: 

Utilising the output 'User Code' above, the taged stories can be retrieved. 

### 3.1 Begin with 'n' tags: Disliked Web articles.

In [10]:
# Parsing out the access token from the 'user.text' string:
access_token = user.text.split('=')[1].split('&username')[0]

access_token

'c3427703-c4f7-a7cc-4470-502f4a'

In [11]:
no_articles_params = {'consumer_key': pocket_API_key, 'access_token': access_token, 'tag': 'n'}

no_articles_result = requests.post('https://getpocket.com/v3/get', data = no_articles_params)

no_articles_result.text

'{"status":1,"complete":1,"list":{"2284795078":{"item_id":"2284795078","resolved_id":"2284795078","given_url":"https:\\/\\/www.hostgator.com\\/blog\\/ways-redirect-website-url\\/","given_title":"5 Ways to Redirect a Website URL - How it Works | HostGator Blog","favorite":"0","status":"0","time_added":"1580874925","time_updated":"1580874930","time_read":"0","time_favorited":"0","sort_id":0,"resolved_title":"5 Ways To Redirect A Website URL","resolved_url":"https:\\/\\/www.hostgator.com\\/blog\\/ways-redirect-website-url\\/","excerpt":"HostGator often offers promotions, coupons and special offers to customers during their initial term. Please note that special offers are limited-time promotional prices that are available to new customers and are valid for the Initial Term only, and not for successive or renewal periods.","is_article":"1","is_index":"0","has_video":"0","has_image":"0","word_count":"145","lang":"en","amp_url":"https:\\/\\/www.hostgator.com\\/blog\\/ways-redirect-website-ur

The output above should be a long JSON string that is related to the 'n' tag. However, at this point, only the URL is of interest.

### 3.2 Create a list of URLs:

In [12]:
no_articles_jf = json.loads(no_articles_result.text)
no_articles_jd = no_articles_jf['list']

In [13]:
# Inspect:
no_articles_jd

{'2284795078': {'item_id': '2284795078',
  'resolved_id': '2284795078',
  'given_url': 'https://www.hostgator.com/blog/ways-redirect-website-url/',
  'given_title': '5 Ways to Redirect a Website URL - How it Works | HostGator Blog',
  'favorite': '0',
  'status': '0',
  'time_added': '1580874925',
  'time_updated': '1580874930',
  'time_read': '0',
  'time_favorited': '0',
  'sort_id': 0,
  'resolved_title': '5 Ways To Redirect A Website URL',
  'resolved_url': 'https://www.hostgator.com/blog/ways-redirect-website-url/',
  'excerpt': 'HostGator often offers promotions, coupons and special offers to customers during their initial term. Please note that special offers are limited-time promotional prices that are available to new customers and are valid for the Initial Term only, and not for successive or renewal periods.',
  'is_article': '1',
  'is_index': '0',
  'has_video': '0',
  'has_image': '0',
  'word_count': '145',
  'lang': 'en',
  'amp_url': 'https://www.hostgator.com/blog/way

In [14]:
no_articles_URL = []
for i in no_articles_jd.values():
    url_dat = i.get('resolved_url')
    no_articles_URL.append(url_dat)

no_articles_URL  

['https://www.hostgator.com/blog/ways-redirect-website-url/',
 'https://au.sports.yahoo.com/australian-open-2020-coach-says-serena-williams-needs-change-213845665.html',
 'https://www.smh.com.au/sport/soccer/liverpool-s-kids-get-job-done-in-fa-cup-20200205-p53xw0.html',
 'https://www.foxsports.com.au/nrl/nrl-premiership/teams/sharks/sharks-meet-with-josh-dugans-agent-as-900k-star-threatened-with-reserve-grade/news-story/ed0b19edc12a0d39336c1869c4656da0',
 'https://www.theaustralian.com.au/sport/nrl/israel-folau-speaks-amid-super-league-clubs-legal-threat/news-story/8005ab670a0f8b80c27d76635658e03f',
 'https://www.gamesradar.com/blizzard-apologizes-for-warcraft-3-reforgeds-troubled-launch/',
 'https://www.theage.com.au/national/australia-s-opioid-crisis-how-pain-management-got-out-of-control-20200204-p53xrn.html',
 'https://www.news.com.au/national/victoria/news/two-men-dead-after-highspeed-chase-ends-in-preston/news-story/aa2b5110b6c8b084cf19c2142ecd84e4',
 'https://www.theguardian.com

As it can be seen, these are the URLs that consist of articles that isn't interesting.

#### Next is to save all these into a DataFrame:

In [15]:
no_URL_df = pd.DataFrame(no_articles_URL, columns = ['urls'])

# Create new column called 'wanted':
no_URL_df = no_URL_df.assign(wanted = lambda x: 'n')

no_URL_df

,urls,wanted
0,https://www.hostgator.com/blog/ways-redirect-website-url/,n
1,https://au.sports.yahoo.com/australian-open-2020-coach-says-serena-williams-needs-change-213845665.html,n
2,https://www.smh.com.au/sport/soccer/liverpool-s-kids-get-job-done-in-fa-cup-20200205-p53xw0.html,n
3,https://www.foxsports.com.au/nrl/nrl-premiership/teams/sharks/sharks-meet-with-josh-dugans-agent-as-900k-star-threatened-with-reserve-grade/news-story/ed0b19edc12a0d39336c1869c4656da0,n
4,https://www.theaustralian.com.au/sport/nrl/israel-folau-speaks-amid-super-league-clubs-legal-threat/news-story/8005ab670a0f8b80c27d76635658e03f,n
5,https://www.gamesradar.com/blizzard-apologizes-for-warcraft-3-reforgeds-troubled-launch/,n
6,https://www.theage.com.au/national/australia-s-opioid-crisis-how-pain-management-got-out-of-control-20200204-p53xrn.html,n
7,https://www.news.com.au/national/victoria/news/two-men-dead-after-highspeed-chase-ends-in-preston/news-story/aa2b5110b6c8b084cf19c2142ecd84e4,n
8,https://www.theguardian.com/australia-news/live/2020/feb/05/national-party-coalition-cabinet-reshuffle-scott-morrison-politics-live,n
9,https://www.notebookcheck.net/No-Android-10-update-for-your-Redmi-Note-8-Pro-You-re-not-alone.452913.0.html,n


### 3.3 Extract the 'y' tags: Liked Web articles.

In [22]:
def get_articles(tag, pocket_API_key, access_token):
    '''
    Func -> Extract article data.
    
    params for this func.:
    tag = 'y' or 'n'
    '''
    articles_params = {'consumer_key': pocket_API_key, 'access_token': access_token, 'tag': tag}
    articles_result = requests.post('https://getpocket.com/v3/get', data = articles_params)
    articles_jf = json.loads(articles_result.text)
    articles_jd = articles_jf['list']
    
    articles_URL = []
    for i in articles_jd.values():
        url_dat = i.get('resolved_url')
        articles_URL.append(url_dat)
        
    URL_df = pd.DataFrame(articles_URL, columns = ['urls'])
    URL_df = URL_df.assign(wanted = lambda x: tag)
    
    return URL_df

In [28]:
yes_URL_df = get_articles('y', pocket_API_key, access_token)

In [29]:
yes_URL_df.head()

,urls,wanted
0,https://twitter.com/acombs,y
1,https://www.fool.com.au/2020/02/05/where-i-would-invest-10000-into-asx-shares-right-now/,y
2,https://www.smh.com.au/world/asia/japan-confirms-10-new-virus-cases-on-ship-as-china-toll-rises-20200205-p53xyt.html,y
3,https://www.notebookcheck.net/New-Galaxy-S20-series-US-pricing-leak-makes-for-some-hard-reading.452842.0.html,y
4,https://www.notebookcheck.net/Intel-s-Core-i9-10900K-is-neck-and-neck-with-AMD-s-Ryzen-9-3900X-in-the-latest-3DMark-tests.452935.0.html,y


### 3.4 Join both the 'n' and 'y' tagged article dataframe:

In [30]:
articles_df = pd.concat([yes_URL_df, no_URL_df])
articles_df.dropna(inplace = True)

In [31]:
articles_df

,urls,wanted
0,https://twitter.com/acombs,y
1,https://www.fool.com.au/2020/02/05/where-i-would-invest-10000-into-asx-shares-right-now/,y
2,https://www.smh.com.au/world/asia/japan-confirms-10-new-virus-cases-on-ship-as-china-toll-rises-20200205-p53xyt.html,y
3,https://www.notebookcheck.net/New-Galaxy-S20-series-US-pricing-leak-makes-for-some-hard-reading.452842.0.html,y
4,https://www.notebookcheck.net/Intel-s-Core-i9-10900K-is-neck-and-neck-with-AMD-s-Ryzen-9-3900X-in-the-latest-3DMark-tests.452935.0.html,y
5,https://www.popularmechanics.com/science/energy/a30337693/powder-nuclear-fusion/,y
6,https://www.theguardian.com/environment/2019/oct/27/nuclear-fusion-research-power-generation-iter-jet-step-carbon-neutral-2050-boris-johnson,y
7,https://www.bbc.com/news/science-environment-50267017,y
8,https://www.macrumors.com/2020/02/04/disney-plus-over-28-million-subscribers/,y
9,https://www.notebookcheck.net/Samsung-debuts-Flashbolt-its-latest-and-highest-capacity-generation-of-HBM2E-memory.452973.0.html,y


# 4 Download the HTML data for each article:

This stage uses the Embedly API to download and extract story bodies.